# Tarea 1: Agente de Generación de Imágenes con Aprobación de Coste

**Objetivo:** Construir un agente que genera imágenes y pide confirmación para solicitudes masivas (>1 imagen).

## Plan de Acción

1.  **Investigación y Selección de la Herramienta (API de Imágenes)**
2.  **Construcción de la Herramienta Base (Función de Generación)**
3.  **Creación de la Lógica del Agente (Flujo de Aprobación)**
4.  **Ensamblaje y Pruebas en este Cuaderno**
5.  **Puente hacia el Framework ADK (Conceptos)**


---
### Paso 1: Investigación y Selección de la Herramienta

Hemos investigado varias APIs de generación de imágenes. Nuestro objetivo era encontrar una que fuera:

- **Accesible:** Preferiblemente con una capa gratuita generosa.
- **Fácil de usar:** Con una buena biblioteca cliente en Python.
- **Bien documentada:** Para poder consultar y aprender rápidamente.

**Decisión:** Para este ejercicio, usaremos la API de **Stability AI**. Su API es directa, tienen buena documentación y suelen ofrecer créditos iniciales que son más que suficientes para nuestra tarea.

Para usarla, necesitarás una API Key. Puedes obtenerla gratis registrándote en [DreamStudio](https://platform.stability.ai/).


---
### Paso 2: Construcción de la Herramienta Base (Función de Generación)

Ahora crearemos la función `generar_una_imagen`. Esta será nuestra "herramienta". Su única responsabilidad es tomar un `prompt` (el texto descriptivo), llamar a la API de Stability AI y guardar la imagen resultante en un archivo.


In [ ]:
# Primero, instalamos las librerías necesarias.
!pip install stability-sdk


In [ ]:
import os
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation
from stability_sdk import client
from getpass import getpass
from PIL import Image
import io

# --- Configuración de la API Key ---
# Para que el cuaderno funcione, necesitas una API key de Stability AI.
# 1. Ve a https://platform.stability.ai/ y regístrate.
# 2. Copia tu API Key.
# 3. Pégala cuando el siguiente campo te lo pida.

api_key = os.environ.get('STABILITY_KEY')
if not api_key:
    try:
        api_key = getpass('Introduce tu API Key de Stability AI: ')
    except (IOError, EOFError):
        print('No se pudo leer la API key.')

# --- Creación del cliente de la API ---
if api_key:
    stability_api = client.StabilityInference(
        key=api_key,
        verbose=True, 
        engine="stable-diffusion-xl-1024-v1-0",
    )
else:
    print("API Key no configurada. No se podrán generar imágenes.")


In [ ]:
def generar_una_imagen(prompt: str, nombre_archivo: str = "output.png"):
    """
    Llama a la API de Stability AI para generar una imagen a partir de un prompt.
    Guarda la imagen generada en un archivo local.
    """
    if 'stability_api' not in globals():
        print("El cliente de la API no está inicializado. Verifica tu API Key.")
        return None

    print(f"\n🤖 Generando imagen para el prompt: '{prompt}'...")
    
    # Llamada a la API
    answers = stability_api.generate(
        prompt=prompt,
        seed=42, # Semilla para reproducibilidad
        steps=40,
        cfg_scale=8.0,
        width=1024,
        height=1024,
        samples=1,
        sampler=generation.SAMPLER_K_DPMPP_2M
    )
    
    # Procesamiento de la respuesta
    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                print("ADVERTENCIA: La solicitud fue filtrada por políticas de seguridad.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                img = Image.open(io.BytesIO(artifact.binary))
                img.save(nombre_archivo)
                print(f"✅ Imagen guardada como '{nombre_archivo}'")
                # Devuelve la ruta del archivo para confirmar el éxito
                return nombre_archivo
    
    # Si no se generó ninguna imagen
    return None

# --- Prueba rápida de la herramienta ---
# Si la API key fue introducida, hacemos una prueba para verificar que todo funciona.
if 'api_key' in locals() and api_key:
    print("--- Realizando prueba de la herramienta 'generar_una_imagen' ---")
    generar_una_imagen("A cat with a wizard hat, watercolor style")


---
### Paso 3: Creación de la Lógica del Agente

¡Nuestra herramienta funciona! Ahora vamos a construir el "cerebro" del agente.

El agente `agente_generador_de_imagenes` no generará las imágenes directamente. En su lugar, orquestará el trabajo: recibirá la solicitud del usuario y decidirá si debe usar la herramienta `generar_una_imagen` inmediatamente o si debe pedir permiso primero.


In [ ]:
def agente_generador_de_imagenes(prompt: str, num_imagenes: int):
    """
    Orquesta la generación de imágenes con una capa de aprobación.
    - Si num_imagenes es 1, genera la imagen directamente.
    - Si num_imagenes > 1, pide confirmación al usuario.
    """
    print(f"\n🧠 Agente ha recibido una solicitud: '{prompt}' (x{num_imagenes})")

    # Lógica de decisión del agente
    if num_imagenes <= 0:
        print("🤔 Número de imágenes no válido. Debe ser 1 o más.")
        return

    if num_imagenes == 1:
        print("✅ Solicitud de una sola imagen. Aprobación automática.")
        generar_una_imagen(prompt, nombre_archivo=f"imagen_unica.png")
    else: # Solicitud masiva
        print(f"⚠️  SOLICITUD MASIVA DETECTADA ({num_imagenes} imágenes).")
        
        # Pedir aprobación humana
        try:
            respuesta = input(f"¿Estás seguro de que quieres generar {num_imagenes} imágenes? (s/n): ").lower()
        except (IOError, EOFError):
            print("\nNo se pudo obtener respuesta. Cancelando solicitud.")
            respuesta = 'n'

        if respuesta == 's' or respuesta == 'si':
            print(f"👍 Aprobado. Procediendo a generar {num_imagenes} imágenes.")
            for i in range(num_imagenes):
                # Generamos un prompt ligeramente diferente para cada imagen para variedad
                prompt_variado = f"{prompt}, imagen {i+1} de {num_imagenes}"
                nombre_archivo = f"imagen_masiva_{i+1}.png"
                generar_una_imagen(prompt_variado, nombre_archivo)
        else:
            print("❌ Solicitud cancelada por el usuario.")


In [ ]:
# --- Paso 4: Ensamblaje y Pruebas en el Cuaderno ---
print("--- Probando el agente con una solicitud ÚNICA ---")
agente_generador_de_imagenes("a robot reading a book in a futurist library", 1)


In [ ]:
print("\n\n--- Probando el agente con una solicitud MASIVA (prueba de cancelación) ---")
# Para esta prueba, responde 'n' o 'no' cuando te pregunte.
agente_generador_de_imagenes("A Martian landscape with two moons", 3)

In [ ]:
print("\n\n--- Probando el agente con una solicitud MASIVA (prueba de aprobación) ---")
# Para esta prueba, responde 's' o 'si' cuando te pregunte.
agente_generador_de_imagenes("An enchanted forest at night, with magical lights", 2)


---
### Paso 5: Puente hacia el Framework ADK (Conceptos)

¡Felicidades! Has construido un agente simple pero funcional.

**¿Cómo se vería esto en el Google ADK?**

1.  **`generar_una_imagen` sería un `Tool`:** Envolveríamos nuestra función en la anotación `@tool` del ADK. El framework se encargaría de exponer la descripción de la función (`docstring`) al modelo de lenguaje (LLM) para que sepa para qué sirve.

2.  **`agente_generador_de_imagenes` sería el `Agent`:** La lógica de decisión que escribimos (el `if/else` y el `input`) viviría dentro del bucle principal de un agente ADK. En lugar de un `input()` simple, el ADK proporciona mecanismos más robustos para la "intervención humana", permitiendo que el agente se detenga, espere una aprobación externa (por ejemplo, desde una interfaz de usuario) y luego reanude su ejecución.

3.  **El LLM podría tomar más decisiones:** En un agente más avanzado, podríamos darle al LLM la capacidad de decidir *si* necesita aprobación. Podríamos instruirle: "Si la solicitud parece costosa o inusual, pide confirmación". El ADK facilita esta interacción entre el LLM, las herramientas y el usuario.

Este ejercicio es la base. Has separado la **herramienta** (la acción) del **agente** (la lógica de orquestación), que es el principio fundamental detrás de los frameworks de agentes modernos como el ADK.
